# Zadania do wykonania

1. Porównaj zapisywanie i odczytywanie kolekcji (100, 10000, 100 000 elementów) za pomocą trzech technik: modułu `pickle`, `parquet` i `xlsx`.

2. Zbadaj przyspieszenie związane z zastosowaniem pamięci podręcznej na wybranych przykładzie funkcji rekurencyjnej (np. ciągu _Fibonacciego_.

3. Napisz program tworzący bazę danych z interfejsem konsolowym. Wymagane są następujące operacje, dodanie wiersza, usunięcie wiersza, zmiana pola wiersza, wyświetlenie opcji. Menu można zorganizować jako odczytywanie parametrów zwróconych przez funkcję `input` lub z pomocą komend (łatwiejszy sposób).

In [150]:
# zadanie 1

import pickle
import pandas

from fastparquet import ParquetFile
from tempfile import mktemp
from os import unlink


dic = dict()

def fill(rangeX):
    for i in range(1, rangeX):
        dic[i]=i*10

def pickleRW():
    try:
        temp_file = mktemp()
        outfile = open(temp_file,'wb')
        pickle.dump(dic,outfile)
        outfile.flush()
        outfile.close()

        outfile = open(temp_file,'rb')
        temp_col = load(outfile)
        print(temp_col)
        outfile.close()
    finally:
        unlink(temp_file)
        
def parquetRW():
    df = pandas.DataFrame.from_dict(dic)
    try:
        temp_file = mktemp()
        df.to_parquet(temp_file, compression='GZIP')
        pf = ParquetFile(temp_file)
        print(f'Readed columns: {pf.columns}')
        print(f'File info: {pf.info}')
    
        df = pf.to_pandas()
    finally:
        unlink(temp_file)
    

fill(10)
pickleRW()

ModuleNotFoundError: No module named 'ParquetFile'

In [89]:
# zadanie 2

import time
from functools import lru_cache, cached_property


# Recursive algorithm for fibbonacci
def fib(n):
    if n <= 1:
        return n
    else:
        return(fib(n-1) + fib(n-2))
        
        
@lru_cache(maxsize=100)
def func_lru_cached(a: int):
    if a <= 1:
        return a
    else:
        return(func_lru_cached(a-1) + func_lru_cached(a-2))        
       

def exec(number):
    start = time.time()
    fib(number)
    end = time.time()
    result = end - start
    print("Time fib normal: ",result,"s")

    start2 = time.time()
    func_lru_cached(number-1)
    end2 = time.time()
    result2 = end2 - start2
    print("Time fib cache: ",end2 - start2,"s")

    print(f'Cache is {result/result2} times faster\n\n')

    
 
print("For 10th element:")
exec(10)

print("For 20th element:")
exec(20)

print("For 30th element:")
exec(30)

print("For 34th element:")
exec(34)

For 10th element:
Time fib normal:  4.100799560546875e-05 s
Time fib cache:  1.52587890625e-05 s
Cache is 2.6875 times faster


For 20th element:
Time fib normal:  0.010749101638793945 s
Time fib cache:  1.5497207641601562e-05 s
Cache is 693.6153846153846 times faster


For 30th element:
Time fib normal:  0.6121826171875 s
Time fib cache:  2.2411346435546875e-05 s
Cache is 27315.744680851065 times faster


For 34th element:
Time fib normal:  3.995593786239624 s
Time fib cache:  8.106231689453125e-06 s
Cache is 492903.9705882353 times faster




In [ ]:
# zadanie 3

from sqlalchemy import create_engine, Column, String, Integer, Boolean
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager
from tempfile import mktemp
from platform import system


temp_db = mktemp(suffix='.sqlite')

print(f'Using {temp_db}')
print(f'--- Table: cities ---')
print(f'--- Columns: name, population, country  ---')

conn_uri_handler = {
    'Windows': f'sqlite:///{temp_db}',
    'Darwin': f'sqlite:////{temp_db}',
    'Linux': f'sqlite:////{temp_db}',
    'Java': f'sqlite:////{temp_db}'
}

engine = create_engine(conn_uri_handler[system()])
Base = declarative_base(bind=engine)

class City(Base):
    __tablename__ = 'cities'
    id=Column(Integer, primary_key=True,autoincrement=True)
    name=Column('name', String(50))
    population=Column('population', Integer)
    country=Column('country', String(50))

Base.metadata.create_all()
Session = sessionmaker(bind=engine)

@contextmanager
def create_session():
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

def provide_session(func):
    def wrapper(*args, **kwargs):
        print(f'\n\tCalling {func.__name__}\n')
        try:
            with create_session() as session:
                args = (*args, session) if args else (session,) 
                return func(*args, **kwargs)
        except Exception as e:
            print(f'\tError found: {e}')

        return None

    return wrapper

@provide_session
def insert(session):
    name2 = input('\tEnter city name: ')
    
    population2 = input('\tEnter city population: ')
    if not population2.isnumeric():
        print('\t It to not numeric value!')
        while not population2.isnumeric():
            population2 = input('\tEnter city population: ')
            print('\t It to not numeric value!')
        
    country2 = input('\tEnter city country: ')
    
    session.add(City(name=name2, population=population2, country=country2))

@provide_session
def display(session):
    i = 0
    city = session.query(City).all()
    for cityN in city:
        i += 1
        print(f'\t{cityN.id} --- {cityN.name} --- {cityN.population} --- {cityN.country}')
        
    if i==0:
        print(f'\tNo records in db!')
        

@provide_session
def delete(session):
    x = input('\tRecord id: ')
    y = session.query(City).get(x)
    session.delete(y)

@provide_session   
def update(session):
    x = input('\tRecord id for update: ')
    z = input('\tColumn name for update: ')
    a = input('\tNew value: ')
    
    if z == 'name':
        session.query(City).filter(City.id == x).\
        update({City.name: a}, synchronize_session=False)
    elif z == 'population':
        session.query(City).filter(City.id == x).\
        update({City.population: a}, synchronize_session=False)
    elif z == 'country':
        session.query(City).filter(City.id == x).\
        update({City.country: a}, synchronize_session=False)

        
def displayMenu1():
    print(' ')
    print('1. Insert into table')
    print('2. Delete from table')
    print('3. Update table')
    print('4. Display table')
    print('5. End program')
        
def displayQuestion():
    p = True
    x = input('\tChoose one of the following option: ')
    if x=='1': insert()
    elif x=='2': delete()
    elif x=='3': update()
    elif x=='4': display()
    elif x=='5': p = False
    else:
        print('\n\tNot recognized option')
    print(' ')
    return p


# main program
while 1:        
    displayMenu1()
    if not displayQuestion():
        print('\tGoodbye!')
        break


Using /tmp/tmp3sahra6o.sqlite
--- Table: cities ---
--- Columns: name, population, country  ---
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 4

	Calling display

	No records in db!
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 1

	Calling insert

	Enter city name: Warsaw
	Enter city population: 1700000
	Enter city country: Poland
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 4

	Calling display

	1 --- Warsaw --- 1700000 --- Poland
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 3

	Calling update

	Record id for update: 1
	Column name for update: population
	New value: 1701000
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table